In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Change the following to match the simulation data

# 7 SNARE data

nrod = 7
rtmd_arr = [2.27]
delta_h = [2.1]
run_list = [[13]]
seed_list = [[902]]
t_hem_list = [[299]]
t_ihem_list = [[0]]
t_fus_list = [[0]]


folder_path = 'E:/fusion_forces/fixed_rad/cut_snare/'

index = 0

path_prefix = folder_path + 'ves_ves_cutsnare_rtmd_ini_%.2f_deltah_%.2f_Nunzip_10.00_lp_0.50_nrod_%d_rod_r_0.38_dves_55.9_tension_0.05_'%(rtmd_arr[index],delta_h[index],nrod)
run_arr = run_list[index]
seed_arr = seed_list[index]
t_hem_arr = t_hem_list[index]
t_ihem_arr = t_ihem_list[index]
t_fus_arr = t_fus_list[index]

In [3]:
dt = 0.068/1000 # timestep in microseconds
frame_dt = 100000 # in steps
check_period = 200 # in steps

# Function Definitions

In [4]:
def get_tmd_pos(run, seed, t_start, t_stop):
    
    linker_up_dir = np.load(path_prefix+'run_%d_dir_up_%d.npy'%(run, seed), allow_pickle=True)
    linker_dn_dir = np.load(path_prefix+'run_%d_dir_dn_%d.npy'%(run, seed), allow_pickle=True)
    
    linker_up_pos = np.load(path_prefix+'run_%d_ct_up_%d.npy'%(run, seed), allow_pickle=True) # in units of sigma
    linker_dn_pos = np.load(path_prefix+'run_%d_ct_dn_%d.npy'%(run, seed), allow_pickle=True) # in units of sigma

    tmd_up_pos = - linker_up_dir + linker_up_pos
    tmd_dn_pos = - linker_dn_dir + linker_dn_pos

    tmd_up_pos = tmd_up_pos[t_start:t_stop, ...]
    tmd_dn_pos = tmd_dn_pos[t_start:t_stop, ...]
    
    tmd_up_pos = tmd_up_pos*0.88
    tmd_dn_pos = tmd_dn_pos*0.88 # in nm

    return tmd_up_pos, tmd_dn_pos

In [5]:
def get_linker_force_cart(run, seed, t_start, t_stop):
   
    # Load linker forces
    linker_up_f_comp = np.load(path_prefix+'run_%d_force_up_%d.npy'%(run, seed), allow_pickle=True)
    linker_dn_f_comp = np.load(path_prefix+'run_%d_force_dn_%d.npy'%(run, seed), allow_pickle=True)
    linker_up_f_comp = linker_up_f_comp[t_start:t_stop, ...]
    linker_dn_f_comp = linker_dn_f_comp[t_start:t_stop, ...]
    linker_up_f_comp = -linker_up_f_comp*0.6*4.1/0.88 # in pN
    linker_dn_f_comp = -linker_dn_f_comp*0.6*4.1/0.88 # in pN
    linker_up_f = np.linalg.norm(linker_up_f_comp, axis=2)
    linker_dn_f = np.linalg.norm(linker_dn_f_comp, axis=2)
    
    return linker_up_f, linker_dn_f, linker_up_f_comp, linker_dn_f_comp # in pN

In [6]:
def get_snare_force_tot(run, seed, t_start, t_stop):
    
    # Load SNARE COM forces (including linker forces)
    snare_data = np.load(path_prefix+'run_%d_snare_force_%d.npy'%(run, seed), allow_pickle=True)
    snare_data = snare_data[t_start:t_stop, ...]
    # Convert SNARE COM forces to pN
    snare_data = snare_data*0.6*4.1/0.88
    
    return snare_data # in pN

In [7]:
def get_snare_entropic_force(snare_data, linker_up_f_comp, linker_dn_f_comp):
    snare_f = snare_data[:] - linker_up_f_comp[:] - linker_dn_f_comp[:] # Compute total force on the SNARE, excluding linker forces (in pN)
    return snare_f

In [8]:
def get_ring_center(snare_ctd_pos):
    # Find the centroid of the SNARE CTDs
    snare_ctd_centroid = np.mean(snare_ctd_pos, axis=1)
    return snare_ctd_centroid

In [9]:
def get_cterm_pos(run, seed, t_start, t_stop):
    
    # Load linker positions for all other snares
    linker_up_pos = np.load(path_prefix+'run_%d_ct_up_%d.npy'%(run, seed), allow_pickle=True) # in units of sigma
    linker_dn_pos = np.load(path_prefix+'run_%d_ct_dn_%d.npy'%(run, seed), allow_pickle=True) # in units of sigma
    linker_up_pos = linker_up_pos[t_start:t_stop, ...]
    linker_dn_pos = linker_dn_pos[t_start:t_stop, ...]

    # Find the mean position of the CTD for each snare
    snare_ctd_pos = (linker_up_pos + linker_dn_pos)/2 * 0.88 # in nm
    snare_ctd_pos_avg = np.mean(snare_ctd_pos, axis=0) # in nm
    
    return snare_ctd_pos, snare_ctd_pos_avg

In [10]:
def get_ring_radius(snare_ctd_pos, snare_ctd_centroid):
    # Compute the distance of each SNARE CTD from the centroid
    snare_r_centroid = snare_ctd_pos - snare_ctd_centroid[:, np.newaxis, :]
    # Compute the magnitude of the distance of each SNARE CTD from the centroid (in nm)
    snare_r_centroid_magn = np.sqrt(snare_r_centroid[..., 0]**2 + snare_r_centroid[..., 1]**2) # in units of sigma
    #snare_r_centroid_magn = snare_r_centroid_magn*0.88 # in nm
    # compute the mean ring radius
    #snare_r_centroid_magn_mean = np.mean(snare_r_centroid_magn, axis=0)
    #snare_r_centroid_magn_std = np.std(snare_r_centroid_magn, axis=0)
    #snare_r_maxmin = np.max(snare_r_centroid_magn, axis=0)/np.min(snare_r_centroid_magn, axis=0)
    #return snare_r_centroid_magn_mean, snare_r_centroid_magn_std, snare_r_maxmin
    return snare_r_centroid_magn

In [11]:
def get_linker_norm(run, seed, t_start, t_stop):
    
    # Load linker direction vectors (in units of sigma)
    linker_up_dir = np.load(path_prefix+'run_%d_dir_up_%d.npy'%(run, seed), allow_pickle=True)
    linker_dn_dir = np.load(path_prefix+'run_%d_dir_dn_%d.npy'%(run, seed), allow_pickle=True)
    linker_up_dir = linker_up_dir[t_start:t_stop, ...]
    linker_dn_dir = linker_dn_dir[t_start:t_stop, ...]
    #print(linker_up_dir.shape)
    #print(linker_dn_dir.shape)
    
    # Compute the linker vector magnitudes (in units of sigma)
    linker_up_magn = np.linalg.norm(linker_up_dir, axis=2, keepdims=True)
    linker_dn_magn = np.linalg.norm(linker_dn_dir, axis=2, keepdims=True)
   
    # Check that both linker_up_dir and linker_dn_dir are pointing from CTD to TMD
    # z_TMD - z_CTD > 0 for the up linkers, so linker_up_dir[2] should be positive
    # z_TMD - z_CTD < 0 for the dn linkers, so linker_dn_dir[2] should be negative
    if np.mean(linker_up_dir[:10,:,2]) < 0: 
        #print('Inverting linker_up_dir')
        linker_up_dir = -linker_up_dir
    if np.mean(linker_dn_dir[:10,:,2]) > 0: 
        #print('Inverting linker_dn_dir')
        linker_dn_dir = -linker_dn_dir

    # Compute the linker unit vectors (pointing from CTD to TMD)
    linker_up_dir_norm = linker_up_dir / (linker_up_magn + 1e-10) # add 1e-10 to avoid division by zero
    linker_dn_dir_norm = linker_dn_dir / (linker_dn_magn + 1e-10) # add 1e-10 to avoid division by zero

    return linker_up_dir_norm, linker_dn_dir_norm

In [12]:
def get_polar_dir(tmd_pos_avg):

    #ori = get_ring_center(tmd_pos_avg)
    ori = np.array([0,0,0])
    # Translate linker COM positions relative to the new origin
    tmd_pos_avg = tmd_pos_avg - ori
    
    # Get unit vectors for the r and theta directions for each SNARE
    rx = tmd_pos_avg[..., 0]/np.sqrt(tmd_pos_avg[..., 0]**2 + tmd_pos_avg[..., 1]**2)
    ry = tmd_pos_avg[..., 1]/np.sqrt(tmd_pos_avg[..., 0]**2 + tmd_pos_avg[..., 1]**2)

    thetax = -ry
    thetay = rx

    return rx, ry, thetax, thetay

In [13]:
def get_polar_force(snare_f_x, snare_f_y, snare_f_z, rx, ry, thetax, thetay):
    
    # Compute the force components in the r and theta directions
    snare_f_r = snare_f_x*rx + snare_f_y*ry
    snare_f_theta = snare_f_x*thetax + snare_f_y*thetay
    return snare_f_r, snare_f_theta, snare_f_z

In [14]:
def get_polar_pos(snare_ctd_pos, rx, ry, thetax, thetay):
    
    # Get unit vectors for the r and theta directions for each SNARE
    r = snare_ctd_pos[..., 0]*rx + snare_ctd_pos[..., 1]*ry
    theta = snare_ctd_pos[..., 0]*thetax + snare_ctd_pos[..., 1]*thetay

    return r, theta

In [15]:
from scipy.signal import correlate

def autocorr(x):
    
    #Normalize the result to have it range from -1 to 1, so the autocorrelation at lag 0 is exactly 1
    norm = np.sum(x ** 2)
    
    # The 'same' mode returns the central part of the correlation
    # that is the same size as the input signal.
    acf = correlate(x, x, mode='same') / norm
    
    # Since the result includes negative lags, we only want the second half
    # which corresponds to the positive lags (including lag 0 at the center)
    acf = acf[acf.size // 2:]
    return acf

# Calculations

In [16]:
snare_f_x_arr_all = []
snare_f_y_arr_all = []
snare_f_z_arr_all = []
snare_data_x_arr_all = []
snare_data_y_arr_all = []
snare_data_z_arr_all = []
linker_up_f_x_arr_all = []
linker_up_f_y_arr_all = []
linker_up_f_z_arr_all = []
linker_dn_f_x_arr_all = []
linker_dn_f_y_arr_all = []
linker_dn_f_z_arr_all = []
linker_up_arr_all = []
linker_dn_arr_all = []
snare_ctd_pos_arr_all = []
snare_ctd_pos_avg_arr = []
tmd_pos_up_arr = []
tmd_pos_dn_arr = []

ring_radius_arr = []
ring_radius_std_arr = []
#ring_radius_maxmin_arr = []
n_measurements_arr = []

In [ ]:
for i in range(len(run_arr)):
    run = run_arr[i]
    seed = seed_arr[i]
    print('run = %d, seed = %d'%(run, seed))

    frame_start = 0
    frame_stop = t_hem_arr[i]

    t_start = int((frame_start)*frame_dt/check_period)
    t_stop = int((frame_stop-1)*frame_dt/check_period)

    linker_up_f_all, linker_dn_f_all, linker_up_f_comp_all, linker_dn_f_comp_all = get_linker_force_cart(run, seed, t_start, t_stop)
    snare_data_all = get_snare_force_tot(run, seed, t_start, t_stop)
    snare_f_all = get_snare_entropic_force(snare_data_all, linker_up_f_comp_all, linker_dn_f_comp_all)
    snare_ctd_pos, snare_ctd_pos_avg = get_cterm_pos(run, seed, t_start, t_stop)
    tmd_pos_up, tmd_pos_dn = get_tmd_pos(run, seed, t_start, t_stop)
    tmd_pos_avg = (tmd_pos_up + tmd_pos_dn)/2
    #ring_radius, ring_radius_std, ring_radius_maxmin = get_ring_radius(snare_ctd_pos, get_ring_center(snare_ctd_pos))
    ring_radius = get_ring_radius(tmd_pos_avg, get_ring_center(tmd_pos_avg))

    n_measurements = np.shape(snare_data_all)[0]
    n_measurements_arr.append(n_measurements)

    snare_f_x_arr_all.append(snare_f_all[..., 0])
    snare_f_y_arr_all.append(snare_f_all[..., 1])
    snare_f_z_arr_all.append(snare_f_all[..., 2])
    snare_data_x_arr_all.append(snare_data_all[..., 0])
    snare_data_y_arr_all.append(snare_data_all[..., 1])
    snare_data_z_arr_all.append(snare_data_all[..., 2])
    linker_up_f_x_arr_all.append(linker_up_f_comp_all[..., 0])
    linker_up_f_y_arr_all.append(linker_up_f_comp_all[..., 1])
    linker_up_f_z_arr_all.append(linker_up_f_comp_all[..., 2])
    linker_dn_f_x_arr_all.append(linker_dn_f_comp_all[..., 0])
    linker_dn_f_y_arr_all.append(linker_dn_f_comp_all[..., 1])
    linker_dn_f_z_arr_all.append(linker_dn_f_comp_all[..., 2])
    linker_up_arr_all.append(linker_up_f_all)
    linker_dn_arr_all.append(linker_dn_f_all)
    snare_ctd_pos_arr_all.append(snare_ctd_pos)
    snare_ctd_pos_avg_arr.append(snare_ctd_pos_avg)
    tmd_pos_up_arr.append(tmd_pos_up)
    tmd_pos_dn_arr.append(tmd_pos_dn)

    ring_radius_arr.append(ring_radius)
    #ring_radius_std_arr.append(ring_radius_std)
    #ring_radius_maxmin_arr.append(ring_radius_maxmin)
    
    del tmd_pos_up, tmd_pos_dn, tmd_pos_avg, snare_ctd_pos, snare_ctd_pos_avg, ring_radius
    del linker_up_f_comp_all, linker_dn_f_comp_all, snare_data_all, snare_f_all, linker_up_f_all, linker_dn_f_all
    #del ring_radius, ring_radius_std, ring_radius_maxmin
    

In [18]:
# Flatten all force arrays into 2D arrays of shape (n_measurements, nsnare) 

snare_f_x_arr_all = np.concatenate(snare_f_x_arr_all, axis=0)
snare_f_y_arr_all = np.concatenate(snare_f_y_arr_all, axis=0)
snare_f_z_arr_all = np.concatenate(snare_f_z_arr_all, axis=0)
snare_data_x_arr_all = np.concatenate(snare_data_x_arr_all, axis=0)
snare_data_y_arr_all = np.concatenate(snare_data_y_arr_all, axis=0)
snare_data_z_arr_all = np.concatenate(snare_data_z_arr_all, axis=0)
linker_up_f_x_arr_all = np.concatenate(linker_up_f_x_arr_all, axis=0)
linker_up_f_y_arr_all = np.concatenate(linker_up_f_y_arr_all, axis=0)
linker_up_f_z_arr_all = np.concatenate(linker_up_f_z_arr_all, axis=0)
linker_dn_f_x_arr_all = np.concatenate(linker_dn_f_x_arr_all, axis=0)
linker_dn_f_y_arr_all = np.concatenate(linker_dn_f_y_arr_all, axis=0)
linker_dn_f_z_arr_all = np.concatenate(linker_dn_f_z_arr_all, axis=0)
linker_up_arr_all = np.concatenate(linker_up_arr_all, axis=0)
linker_dn_arr_all = np.concatenate(linker_dn_arr_all, axis=0)
snare_ctd_pos_arr_all = np.concatenate(snare_ctd_pos_arr_all, axis=0)
tmd_pos_up_arr = np.concatenate(tmd_pos_up_arr, axis=0)
tmd_pos_dn_arr = np.concatenate(tmd_pos_dn_arr, axis=0)
ring_radius_arr = np.concatenate(ring_radius_arr, axis=0)

In [19]:
# Compute total linker forces (sum of up and down linkers)
linker_f_x_arr_all = linker_up_f_x_arr_all + linker_dn_f_x_arr_all
linker_f_y_arr_all = linker_up_f_y_arr_all + linker_dn_f_y_arr_all
linker_f_z_arr_all = linker_up_f_z_arr_all + linker_dn_f_z_arr_all

In [ ]:
# Compute averages and standard deviations ISNARE

corr = 1
n_measurements = np.sum(n_measurements_arr) # total number of measurements
print('Number of measurements per SNARE = %d'%n_measurements)
print('Measurement time: %.2f us'%(n_measurements*dt*check_period))

## AVERAGES OVER ALL SNARES

### COM forces

In [ ]:
print('COM force x')
print('Mean: %.2f pN'%np.mean(snare_data_x_arr_all))
print('SD: %.2f pN'%np.std(snare_data_x_arr_all))
print('SEM: %.2f pN'%(np.std(snare_data_x_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(snare_data_x_arr_all))

print('COM force y')
print('Mean: %.2f pN'%np.mean(snare_data_y_arr_all))
print('SD: %.2f pN'%np.std(snare_data_y_arr_all))
print('SEM: %.2f pN'%(np.std(snare_data_y_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(snare_data_y_arr_all))

print('COM force z')
print('Mean: %.2f pN'%np.mean(snare_data_z_arr_all))
print('SD: %.2f pN'%np.std(snare_data_z_arr_all))
print('SEM: %.2f pN'%(np.std(snare_data_z_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(snare_data_z_arr_all))

### Wall forces

In [ ]:
print('Wall force f_x')
print('Mean: %.2f pN'%np.mean(snare_f_x_arr_all))
print('SD: %.2f pN'%np.std(snare_f_x_arr_all))
print('SEM: %.2f pN'%(np.std(snare_f_x_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(snare_f_x_arr_all))

print('Wall force f_y')
print('Mean: %.2f pN'%np.mean(snare_f_y_arr_all))
print('SD: %.2f pN'%np.std(snare_f_y_arr_all))
print('SEM: %.2f pN'%(np.std(snare_f_y_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(snare_f_y_arr_all))

print('Wall force f_z')
print('Mean: %.2f pN'%np.mean(snare_f_z_arr_all))
print('SD: %.2f pN'%np.std(snare_f_z_arr_all))
print('SEM: %.2f pN'%(np.std(snare_f_z_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(snare_f_z_arr_all))

### Linker forces

In [ ]:
print('x up linker force T_x')
print('Mean: %.2f pN'%np.mean(linker_up_f_x_arr_all))
print('SD: %.2f pN'%np.std(linker_up_f_x_arr_all))
print('SEM: %.2f pN'%(np.std(linker_up_f_x_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_up_f_x_arr_all))

print('y up linker force T_y')
print('Mean: %.2f pN'%np.mean(linker_up_f_y_arr_all))
print('SD: %.2f pN'%np.std(linker_up_f_y_arr_all))
print('SEM: %.2f pN'%(np.std(linker_up_f_y_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_up_f_y_arr_all))

print('z up linker force T_z')
print('Mean: %.2f pN'%np.mean(linker_up_f_z_arr_all))
print('SD: %.2f pN'%np.std(linker_up_f_z_arr_all))
print('SEM: %.2f pN'%(np.std(linker_up_f_z_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_up_f_z_arr_all))

print('x down linker force T_x')
print('Mean: %.2f pN'%np.mean(linker_dn_f_x_arr_all))
print('SD: %.2f pN'%np.std(linker_dn_f_x_arr_all))
print('SEM: %.2f pN'%(np.std(linker_dn_f_x_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_dn_f_x_arr_all))

print('y down linker force T_y')
print('Mean: %.2f pN'%np.mean(linker_dn_f_y_arr_all))
print('SD: %.2f pN'%np.std(linker_dn_f_y_arr_all))
print('SEM: %.2f pN'%(np.std(linker_dn_f_y_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_dn_f_y_arr_all))

print('z down linker force T_z')
print('Mean: %.2f pN'%np.mean(linker_dn_f_z_arr_all))
print('SD: %.2f pN'%np.std(linker_dn_f_z_arr_all))
print('SEM: %.2f pN'%(np.std(linker_dn_f_z_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_dn_f_z_arr_all))

print('x both linker forces T_x')
print('Mean: %.2f pN'%np.mean(linker_f_x_arr_all))
print('SD: %.2f pN'%np.std(linker_f_x_arr_all))
print('SEM: %.2f pN'%(np.std(linker_f_x_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_f_x_arr_all))

print('y both linker forces T_y')
print('Mean: %.2f pN'%np.mean(linker_f_y_arr_all))
print('SD: %.2f pN'%np.std(linker_f_y_arr_all))
print('SEM: %.2f pN'%(np.std(linker_f_y_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_f_y_arr_all))

print('z both linker forces T_z')
print('Mean: %.2f pN'%np.mean(linker_f_z_arr_all))
print('SD: %.2f pN'%np.std(linker_f_z_arr_all))
print('SEM: %.2f pN'%(np.std(linker_f_z_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_f_z_arr_all))

print('Mean zippering force up T_up')
print('Mean: %.2f pN'%np.mean(linker_up_arr_all))
print('SD: %.2f pN'%np.std(linker_up_arr_all))
print('SEM: %.2f pN'%(np.std(linker_up_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_up_arr_all))

print('Mean zippering force down T_dn')
print('Mean: %.2f pN'%np.mean(linker_dn_arr_all))
print('SD: %.2f pN'%np.std(linker_dn_arr_all))
print('SEM: %.2f pN'%(np.std(linker_dn_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_dn_arr_all))

## AVERAGES OVER EACH SNARE

### COM forces

In [ ]:
'''
print('COM force x')
print('Mean (pN): ', np.mean(snare_data_x_arr_all, axis=0))
print('SD (pN): ', np.std(snare_data_x_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_data_x_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_data_x_arr_all, axis=0))

print('\nCOM force y')
print('Mean (pN): ', np.mean(snare_data_y_arr_all, axis=0))
print('SD (pN): ', np.std(snare_data_y_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_data_y_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_data_y_arr_all, axis=0))

print('\nCOM force z')
print('Mean (pN): ', np.mean(snare_data_z_arr_all, axis=0))
print('SD (pN): ', np.std(snare_data_z_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_data_z_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_data_z_arr_all, axis=0))
'''

### Wall forces

In [ ]:
'''
print('Wall force f_x')
print('Mean (pN): ', np.mean(snare_f_x_arr_all, axis=0))
print('SD (pN): ', np.std(snare_f_x_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_f_x_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_f_x_arr_all, axis=0))

print('Wall force f_y')
print('Mean (pN): ', np.mean(snare_f_y_arr_all, axis=0))
print('SD (pN): ', np.std(snare_f_y_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_f_y_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_f_y_arr_all, axis=0))

print('Wall force f_z')
print('Mean (pN): ', np.mean(snare_f_z_arr_all, axis=0))
print('SD (pN): ', np.std(snare_f_z_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_f_z_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_f_z_arr_all, axis=0))
'''

### Linker forces

In [ ]:
'''
print('x up linker force T_x')
print('Mean (pN): ', np.mean(linker_up_f_x_arr_all, axis=0))
print('SD (pN): ', np.std(linker_up_f_x_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_up_f_x_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_up_f_x_arr_all, axis=0))

print('y up linker force T_y')
print('Mean (pN): ', np.mean(linker_up_f_y_arr_all, axis=0))
print('SD (pN): ', np.std(linker_up_f_y_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_up_f_y_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_up_f_y_arr_all, axis=0))

print('z up linker force T_z')
print('Mean (pN): ', np.mean(linker_up_f_z_arr_all, axis=0))
print('SD (pN): ', np.std(linker_up_f_z_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_up_f_z_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_up_f_z_arr_all, axis=0))

print('x down linker force T_x')
print('Mean (pN): ', np.mean(linker_dn_f_x_arr_all, axis=0))
print('SD (pN): ', np.std(linker_dn_f_x_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_dn_f_x_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_dn_f_x_arr_all, axis=0))

print('y down linker force T_y')
print('Mean (pN): ', np.mean(linker_dn_f_y_arr_all, axis=0))
print('SD (pN): ', np.std(linker_dn_f_y_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_dn_f_y_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_dn_f_y_arr_all, axis=0))

print('z down linker force T_z')
print('Mean (pN): ', np.mean(linker_dn_f_z_arr_all, axis=0))
print('SD (pN): ', np.std(linker_dn_f_z_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_dn_f_z_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_dn_f_z_arr_all, axis=0))

print('x both linker forces T_x')
print('Mean (pN): ', np.mean(linker_f_x_arr_all, axis=0))
print('SD (pN): ', np.std(linker_f_x_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_f_x_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_f_x_arr_all, axis=0))

print('y both linker forces T_y')
print('Mean (pN): ', np.mean(linker_f_y_arr_all, axis=0))
print('SD (pN): ', np.std(linker_f_y_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_f_y_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_f_y_arr_all, axis=0))

print('z both linker forces T_z')
print('Mean (pN): ', np.mean(linker_f_z_arr_all, axis=0))
print('SD (pN): ', np.std(linker_f_z_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_f_z_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_f_z_arr_all, axis=0))

print('Mean zippering force up T_up')
print('Mean (pN): ', np.mean(linker_up_arr_all, axis=0))
print('SD (pN): ', np.std(linker_up_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_up_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_up_arr_all, axis=0))

print('Mean zippering force down T_dn')
print('Mean (pN): ', np.mean(linker_dn_arr_all, axis=0))
print('SD (pN): ', np.std(linker_dn_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_dn_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_dn_arr_all, axis=0))
'''

In [ ]:
# Calculate mean linker force magnitude in the xy plane
linker_f_xy_arr_all = np.sqrt(linker_f_x_arr_all**2 + linker_f_y_arr_all**2)
print(np.shape(linker_f_xy_arr_all))
linker_f_xy_arr_avg = np.mean(linker_f_xy_arr_all, axis=0)
print(np.shape(linker_f_xy_arr_avg))
print('Mean linker force magnitude in the xy plane')
print('Mean (pN): ', linker_f_xy_arr_avg)

In [ ]:
# Mean CTD positions
snare_ctd_pos_avg_arr = np.mean(snare_ctd_pos_avg_arr, axis=0)
ori = get_ring_center(snare_ctd_pos_avg_arr)
print(ori)

In [ ]:
# Plot the unit vectors of each snare's linker forces
font_title = {  'weight': 'bold',
                'size': 10}
font_axis = {   'weight': 'normal',
                'size': 10}

plt.figure()
c_arr = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'b', 'g']
for i in range(np.shape(linker_f_x_arr_all)[1]):
    plt.quiver(snare_ctd_pos_avg_arr[i,0], snare_ctd_pos_avg_arr[i,1], np.mean(linker_f_x_arr_all[:, i]), np.mean(linker_f_y_arr_all[:, i]), angles='xy', scale_units='xy', scale=1.5, color=c_arr[i], label='SNARE %d'%(i+1), width=0.02)
plt.plot(ori[0], ori[1], 'kx', markersize=5)
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.xlabel('x (nm)', fontdict=font_axis)
plt.ylabel('y (nm)', fontdict=font_axis)
# Remove tick labels
#plt.xticks([])
#plt.yticks([])
plt.legend()
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Total linker force (x-y projection)')
#plt.title('Top linker force directions')
#plt.title('Bottom linker force directions')
plt.show()

# FIXED POLAR COORDINATES

In [ ]:
# Calculate the linker force magnitude in the xy plane along the mean direction
linker_f_xy_magn_avg = []
for i in range(np.shape(linker_f_x_arr_all)[1]):
    linker_f_xy_magn_avg.append(np.sqrt(np.mean(linker_f_x_arr_all[:, i])**2 + np.mean(linker_f_y_arr_all[:, i])**2))
print('Linker force magnitude in the mean direction in xy plane')
print('Mean (pN): ',linker_f_xy_magn_avg)

In [ ]:
# Get the unit vectors of the linker forces in the radial and theta directions
'''
ld_dir_rad_x = -np.mean(linker_f_x_arr_all, axis = 0)/linker_f_xy_magn_avg
ld_dir_rad_y = -np.mean(linker_f_y_arr_all, axis = 0)/linker_f_xy_magn_avg
ld_dir_rad_magn = np.sqrt(ld_dir_rad_x**2 + ld_dir_rad_y**2)
print('Linker direction in the radial direction')
print('Mean (pN): ', ld_dir_rad_magn)
ld_dir_theta_x = -ld_dir_rad_y
ld_dir_theta_y = ld_dir_rad_x
'''

In [ ]:
# Find the mean position of each SNARE's TMD in cartesian coords
tmd_pos_avg_arr = (tmd_pos_up_arr + tmd_pos_dn_arr)/2
tmd_pos_avg = np.mean(tmd_pos_avg_arr, axis=0)
print(np.shape(tmd_pos_avg_arr))
print(np.shape(tmd_pos_avg))

In [33]:
# Get radial and theta unit vectors pointing from the ring center to the TMDs
ld_dir_rad_x, ld_dir_rad_y, ld_dir_theta_x, ld_dir_theta_y = get_polar_dir(tmd_pos_avg_arr)

In [ ]:
# Plot the unit vectors of each snare's linker forces
plt.figure()
c_arr = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'b', 'g']
for i in range(np.shape(linker_f_x_arr_all)[1]):
    plt.quiver(snare_ctd_pos_avg_arr[i,0], snare_ctd_pos_avg_arr[i,1], np.mean(linker_f_x_arr_all[:, i]), np.mean(linker_f_y_arr_all[:, i]), angles='xy', scale_units='xy', scale=1.5, color=c_arr[i], label='SNARE %d'%(i+1), width=0.02)
    #plt.quiver(tmd_pos_avg[i, 0], tmd_pos_avg[i, 1], np.mean(ld_dir_rad_x[i]), np.mean(ld_dir_rad_y[i]), angles='xy', scale_units='xy', scale=0.5, color=c_arr[i], label='SNARE %d'%(i+1))
    plt.plot(tmd_pos_avg[i, 0], tmd_pos_avg[i, 1], 'x', markersize=5, color='k')
#plt.plot(ori[0], ori[1], 'kx', markersize=5)
#plt.plot(0, 0, 'k.', markersize=8)
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.xlabel('x (nm)', fontdict=font_axis)
plt.ylabel('y (nm)', fontdict=font_axis)
# Remove tick labels
#plt.xticks([])
#plt.yticks([])
#plt.legend()
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Total linker force (x-y projection)')
#plt.title('Top linker force directions')
#plt.title('Bottom linker force directions')
plt.show()

In [35]:
# Convert all forces to polar coordinates
snare_data_rad_arr_all, snare_data_theta_arr_all, snare_data_z_arr_all = get_polar_force(snare_data_x_arr_all, snare_data_y_arr_all, snare_data_z_arr_all, ld_dir_rad_x, ld_dir_rad_y, ld_dir_theta_x, ld_dir_theta_y)
snare_f_rad_arr_all, snare_f_theta_arr_all, snare_f_z_arr_all = get_polar_force(snare_f_x_arr_all, snare_f_y_arr_all, snare_f_z_arr_all, ld_dir_rad_x, ld_dir_rad_y, ld_dir_theta_x, ld_dir_theta_y)
linker_up_f_rad_arr_all, linker_up_f_theta_arr_all, linker_up_f_z_arr_all = get_polar_force(linker_up_f_x_arr_all, linker_up_f_y_arr_all, linker_up_f_z_arr_all, ld_dir_rad_x, ld_dir_rad_y, ld_dir_theta_x, ld_dir_theta_y)
linker_dn_f_rad_arr_all, linker_dn_f_theta_arr_all, linker_dn_f_z_arr_all = get_polar_force(linker_dn_f_x_arr_all, linker_dn_f_y_arr_all, linker_dn_f_z_arr_all, ld_dir_rad_x, ld_dir_rad_y, ld_dir_theta_x, ld_dir_theta_y)

In [36]:
# Compute total linker forces (sum of up and down linkers)
linker_f_rad_arr_all = linker_up_f_rad_arr_all + linker_dn_f_rad_arr_all
linker_f_theta_arr_all = linker_up_f_theta_arr_all + linker_dn_f_theta_arr_all
linker_f_z_arr_all = linker_up_f_z_arr_all + linker_dn_f_z_arr_all

## AVERAGES OVER ALL SNARES

### COM forces

In [ ]:
print('COM force rad')
print('Mean: %.2f pN'%np.mean(snare_data_rad_arr_all))
print('SD: %.2f pN'%np.std(snare_data_rad_arr_all))
print('SEM: %.2f pN'%(np.std(snare_data_rad_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(snare_data_rad_arr_all))

print('\nCOM force theta')
print('Mean: %.2f pN'%np.mean(snare_data_theta_arr_all))
print('SD: %.2f pN'%np.std(snare_data_theta_arr_all))
print('SEM: %.2f pN'%(np.std(snare_data_theta_arr_all)/np.sqrt(n_measurements/corr)))  
print('Median: %.2f pN'%np.median(snare_data_theta_arr_all))

print('\nCOM force z')
print('Mean: %.2f pN'%np.mean(snare_data_z_arr_all))
print('SD: %.2f pN'%np.std(snare_data_z_arr_all))
print('SEM: %.2f pN'%(np.std(snare_data_z_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(snare_data_z_arr_all))

### Wall forces

In [ ]:
print('Wall force f_rad')
print('Mean: %.2f pN'%np.mean(snare_f_rad_arr_all))
print('SD: %.2f pN'%np.std(snare_f_rad_arr_all))
print('SEM: %.2f pN'%(np.std(snare_f_rad_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(snare_f_rad_arr_all))

print('\nWall force f_theta')
print('Mean: %.2f pN'%np.mean(snare_f_theta_arr_all))
print('SD: %.2f pN'%np.std(snare_f_theta_arr_all))
print('SEM: %.2f pN'%(np.std(snare_f_theta_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(snare_f_theta_arr_all))

print('\nWall force f_z')
print('Mean: %.2f pN'%np.mean(snare_f_z_arr_all))
print('SD: %.2f pN'%np.std(snare_f_z_arr_all))
print('SEM: %.2f pN'%(np.std(snare_f_z_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(snare_f_z_arr_all))

### Linker forces

In [ ]:
# Up linker forces
print('rad up linker force T_rad')
print('Mean: %.2f pN'%np.mean(linker_up_f_rad_arr_all))
print('SD: %.2f pN'%np.std(linker_up_f_rad_arr_all))
print('SEM: %.2f pN'%(np.std(linker_up_f_rad_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_up_f_rad_arr_all))

print('\ntheta up linker force T_theta')
print('Mean: %.2f pN'%np.mean(linker_up_f_theta_arr_all))
print('SD: %.2f pN'%np.std(linker_up_f_theta_arr_all))
print('SEM: %.2f pN'%(np.std(linker_up_f_theta_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_up_f_theta_arr_all))

print('\nz up linker force T_z')
print('Mean: %.2f pN'%np.mean(linker_up_f_z_arr_all))
print('SD: %.2f pN'%np.std(linker_up_f_z_arr_all))
print('SEM: %.2f pN'%(np.std(linker_up_f_z_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_up_f_z_arr_all))

# Down linker forces
print('\nrad down linker force T_rad')
print('Mean: %.2f pN'%np.mean(linker_dn_f_rad_arr_all))
print('SD: %.2f pN'%np.std(linker_dn_f_rad_arr_all))
print('SEM: %.2f pN'%(np.std(linker_dn_f_rad_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_dn_f_rad_arr_all))

print('\ntheta down linker force T_theta')
print('Mean: %.2f pN'%np.mean(linker_dn_f_theta_arr_all))
print('SD: %.2f pN'%np.std(linker_dn_f_theta_arr_all))
print('SEM: %.2f pN'%(np.std(linker_dn_f_theta_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_dn_f_theta_arr_all))

print('\nz down linker force T_z')
print('Mean: %.2f pN'%np.mean(linker_dn_f_z_arr_all))
print('SD: %.2f pN'%np.std(linker_dn_f_z_arr_all))
print('SEM: %.2f pN'%(np.std(linker_dn_f_z_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_dn_f_z_arr_all))

# Both linker forces
print('\nrad both linker forces T_rad')
print('Mean: %.2f pN'%np.mean(linker_f_rad_arr_all))
print('SD: %.2f pN'%np.std(linker_f_rad_arr_all))
print('SEM: %.2f pN'%(np.std(linker_f_rad_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_f_rad_arr_all))

print('\ntheta both linker forces T_theta')
print('Mean: %.2f pN'%np.mean(linker_f_theta_arr_all))
print('SD: %.2f pN'%np.std(linker_f_theta_arr_all))
print('SEM: %.2f pN'%(np.std(linker_f_theta_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_f_theta_arr_all))

print('\nz both linker forces T_z')
print('Mean: %.2f pN'%np.mean(linker_f_z_arr_all))
print('SD: %.2f pN'%np.std(linker_f_z_arr_all))
print('SEM: %.2f pN'%(np.std(linker_f_z_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_f_z_arr_all))

# Zippering forces
print('\nMean zippering force up T_up')
print('Mean: %.2f pN'%np.mean(linker_up_arr_all))
print('SD: %.2f pN'%np.std(linker_up_arr_all))
print('SEM: %.2f pN'%(np.std(linker_up_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_up_arr_all))

print('\nMean zippering force down T_dn')
print('Mean: %.2f pN'%np.mean(linker_dn_arr_all))
print('SD: %.2f pN'%np.std(linker_dn_arr_all))
print('SEM: %.2f pN'%(np.std(linker_dn_arr_all)/np.sqrt(n_measurements/corr)))
print('Median: %.2f pN'%np.median(linker_dn_arr_all))

## AVERAGES OVER EACH SNARE

### COM forces

In [ ]:
'''
print('COM force rad')
print('Mean (pN): ', np.mean(snare_data_rad_arr_all, axis=0))
print('SD (pN): ', np.std(snare_data_rad_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_data_rad_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_data_rad_arr_all, axis=0))

print('\nCOM force theta')
print('Mean (pN): ', np.mean(snare_data_theta_arr_all, axis=0))
print('SD (pN): ', np.std(snare_data_theta_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_data_theta_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_data_theta_arr_all, axis=0))

print('\nCOM force z')
print('Mean (pN): ', np.mean(snare_data_z_arr_all, axis=0))
print('SD (pN): ', np.std(snare_data_z_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_data_z_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_data_z_arr_all, axis=0))
'''

### Wall forces

In [ ]:
'''
print('Wall force f_rad')
print('Mean (pN): ', np.mean(snare_f_rad_arr_all, axis=0))
print('SD (pN): ', np.std(snare_f_rad_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_f_rad_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_f_rad_arr_all, axis=0))

print('\nWall force f_theta')
print('Mean (pN): ', np.mean(snare_f_theta_arr_all, axis=0))
print('SD (pN): ', np.std(snare_f_theta_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_f_theta_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_f_theta_arr_all, axis=0))

print('\nWall force f_z')
print('Mean (pN): ', np.mean(snare_f_z_arr_all, axis=0))
print('SD (pN): ', np.std(snare_f_z_arr_all, axis=0))
print('SEM (pN): ', (np.std(snare_f_z_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(snare_f_z_arr_all, axis=0))
'''

### Linker forces

In [ ]:
'''
print('\nrad up linker force T_rad')
print('Mean (pN): ', np.mean(linker_up_f_rad_arr_all, axis=0))
print('SD (pN): ', np.std(linker_up_f_rad_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_up_f_rad_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_up_f_rad_arr_all, axis=0))

print('\ntheta up linker force T_theta')
print('Mean (pN): ', np.mean(linker_up_f_theta_arr_all, axis=0))
print('SD (pN): ', np.std(linker_up_f_theta_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_up_f_theta_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_up_f_theta_arr_all, axis=0))

print('\nz up linker force T_z')
print('Mean (pN): ', np.mean(linker_up_f_z_arr_all, axis=0))
print('SD (pN): ', np.std(linker_up_f_z_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_up_f_z_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_up_f_z_arr_all, axis=0))

print('\nrad down linker force T_rad')
print('Mean (pN): ', np.mean(linker_dn_f_rad_arr_all, axis=0))
print('SD (pN): ', np.std(linker_dn_f_rad_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_dn_f_rad_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_dn_f_rad_arr_all, axis=0))

print('\ntheta down linker force T_theta')
print('Mean (pN): ', np.mean(linker_dn_f_theta_arr_all, axis=0))
print('SD (pN): ', np.std(linker_dn_f_theta_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_dn_f_theta_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_dn_f_theta_arr_all, axis=0))

print('\nz down linker force T_z')
print('Mean (pN): ', np.mean(linker_dn_f_z_arr_all, axis=0))
print('SD (pN): ', np.std(linker_dn_f_z_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_dn_f_z_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_dn_f_z_arr_all, axis=0))

print('\nrad both linker forces T_rad')
print('Mean (pN): ', np.mean(linker_f_rad_arr_all, axis=0))
print('SD (pN): ', np.std(linker_f_rad_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_f_rad_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_f_rad_arr_all, axis=0))

print('\ntheta both linker forces T_theta')
print('Mean (pN): ', np.mean(linker_f_theta_arr_all, axis=0))
print('SD (pN): ', np.std(linker_f_theta_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_f_theta_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_f_theta_arr_all, axis=0))

print('\nz both linker forces T_z')
print('Mean (pN): ', np.mean(linker_f_z_arr_all, axis=0))
print('SD (pN): ', np.std(linker_f_z_arr_all, axis=0))
print('SEM (pN): ', (np.std(linker_f_z_arr_all, axis=0)/np.sqrt(n_measurements/corr)))
print('Median (pN): ', np.median(linker_f_z_arr_all, axis=0))
'''

In [43]:
# Get the C-term position in cartesial coords relative to the TMD position
#snare_ctd_pos_arr_all_rel_tmd = snare_ctd_pos_arr_all - tmd_pos_avg_arr

In [44]:
# Convert the C terminal positions to polar coordinates given the unit vector in the radial direction
#snare_ctd_pos_rad_arr_all, snare_ctd_pos_theta_arr_all = get_polar_pos(snare_ctd_pos_arr_all_rel_tmd, ld_dir_rad_x, ld_dir_rad_y, ld_dir_theta_x, ld_dir_theta_y) 

# SAVE DATA

In [45]:
ring_rad_avg = np.mean(ring_radius_arr)
ring_rad_std = np.std(ring_radius_arr)

T_zip_avg = np.mean((linker_up_arr_all+linker_dn_arr_all)/2) 
T_zip_std = np.std((linker_up_arr_all+linker_dn_arr_all)/2)

F_squeeze_up = np.sum(linker_up_f_z_arr_all, axis = 1)
F_squeeze_dn = np.sum(linker_dn_f_z_arr_all, axis = 1)
F_squeeze_avg = np.mean((F_squeeze_up - F_squeeze_dn)/2)
F_squeeze_std = np.std((F_squeeze_up - F_squeeze_dn)/2)

f_squeeze_avg = np.mean((linker_up_f_z_arr_all - linker_dn_f_z_arr_all)/2)
f_squeeze_std = np.std((linker_up_f_z_arr_all - linker_dn_f_z_arr_all)/2)

T_rad_tot_avg = np.mean(linker_f_rad_arr_all)
T_rad_tot_std = np.std(linker_f_rad_arr_all)
T_theta_tot_avg = np.mean(linker_f_theta_arr_all)
T_theta_tot_std = np.std(linker_f_theta_arr_all)

fent_rad_avg = np.mean(snare_f_rad_arr_all)
fent_rad_std = np.std(snare_f_rad_arr_all)
fent_theta_avg = np.mean(snare_f_theta_arr_all)
fent_theta_std = np.std(snare_f_theta_arr_all)
fent_z_avg = np.mean(snare_f_z_arr_all)
fent_z_std = np.std(snare_f_z_arr_all)

In [ ]:
print(ring_rad_avg)

In [46]:
# Save the data to a .txt file
'''
fname = 'force_data_fixed_coord_nrod_%d_rtmd_%.2f.txt'%(nrod, rtmd_arr[index])

with open(fname, 'w') as file:
    file.write("Number of SNAREs\n")
    file.write(f"{nrod}\n")
    file.write("TMD ring constraint (nm)\n")
    file.write(f"{rtmd_arr[index]*0.88}\n")
    file.write("Number of measurements per SNARE\n")
    file.write(f"{n_measurements}\n")
    file.write("TMD ring radius (nm)\n")
    file.write(f"{ring_rad_avg}\n")
    file.write(f"{ring_rad_std}\n")
    file.write("Zippering force per LD (pN)\n")
    file.write(f"{T_zip_avg}\n")
    file.write(f"{T_zip_std}\n")
    file.write("Total squeezing force (pN)\n")
    file.write(f"{F_squeeze_avg}\n")
    file.write(f"{F_squeeze_std}\n")
    file.write("Squeezing force per LD (pN)\n")
    file.write(f"{f_squeeze_avg}\n")
    file.write(f"{f_squeeze_std}\n")
    file.write("Radial linker force (both LDs) (pN)\n")
    file.write(f"{T_rad_tot_avg}\n")
    file.write(f"{T_rad_tot_std}\n")
    file.write("Theta linker force (both LDs) (pN)\n")
    file.write(f"{T_theta_tot_avg}\n")
    file.write(f"{T_theta_tot_std}\n")
    file.write("Radial entropic force (pN)\n")
    file.write(f"{fent_rad_avg}\n")
    file.write(f"{fent_rad_std}\n")
    file.write("Theta entropic force (pN)\n")
    file.write(f"{fent_theta_avg}\n")
    file.write(f"{fent_theta_std}\n")
    file.write("Z entropic force (pN)\n")
    file.write(f"{fent_z_avg}\n")
    file.write(f"{fent_z_std}\n")
'''

# PLOTS

In [ ]:
# Plot settings
interval = 1 # plotting interval;
#t_plot = np.arange(t_start, t_stop-2900)*dt_save*0.068*1e-3 # in microseconds
t_plot = np.arange(0, linker_f_rad_arr_all.shape[0])*check_period*0.068*1e-3 # in microseconds
print(t_plot.shape)

In [48]:
isnare = 5

In [ ]:
# Plot the mean TMD ring radius
ring_radius_arr_avg = np.mean(ring_radius_arr, axis=1)
ring_rad_interval = np.std(ring_radius_arr, axis=1)

plt.figure()
plt.plot(t_plot, ring_radius_arr_avg, color='g')
plt.fill_between(t_plot, ring_radius_arr_avg - ring_rad_interval, ring_radius_arr_avg + ring_rad_interval, color='g', alpha=0.3, edgecolor='none')
plt.xlabel('Time ($\mu$s)', fontdict=font_axis)
plt.ylabel('TMD ring radius(nm)', fontdict=font_axis)
#plt.title('Ring radius', fontdict=font_title)
#plt.xlim(0,1)
plt.ylim(0,8)
#plt.legend()
plt.show()


In [ ]:
'''
plt.plot(snare_ctd_pos_rad_arr_all[:, isnare], linker_f_rad_arr_all[:, isnare], 'o', markersize=1)
plt.xlabel('r (nm)', fontdict=font_axis)
plt.ylabel('Radial linker force (pN)', fontdict=font_axis)
plt.title('Radial linker force vs radial C-term position')
plt.xlim(-1, 2.5)
plt.ylim(-80, 20)
'''

In [ ]:
# Plot a 2D histogram of the radial linker force vs radial position
'''
plt.figure()
counts2, x_edges2, y_edges2, patches2 = plt.hist2d(snare_ctd_pos_rad_arr_all[:, isnare], linker_f_rad_arr_all[:, isnare], bins=(np.arange(-100, 100, 0.02), np.arange(-100, 100, 1)), cmap='viridis', density=True)
plt.xlabel('r (nm)', fontdict=font_axis)
plt.ylabel('Radial linker force (pN)', fontdict=font_axis)
plt.ylim(-30,10)
plt.xlim(-1,2.5)
plt.title('Radial linker force vs radial C-term position')
# Set equal aspect ratio
#plt.gca().set_aspect('equal', adjustable='box')
cbar = plt.colorbar()
cbar.set_label('Probability density')
'''

In [ ]:
# Plot the radial linker force distribution for one snare
'''
#plt.hist(snare_f_rad_arr_all[:,isnare], bins=np.arange(-100,101,2), density=True, alpha=0.6, color='g', label='Radial SNARE force')
#plt.hist(-(linker_up_f_rad_arr_all[:,isnare]), bins=np.arange(-100,101,2), density=True, alpha=0.4, color='b', label='Top radial linker force')
#plt.hist(-(linker_dn_f_rad_arr_all[:,isnare]), bins=np.arange(-100,101,2), density=True, alpha=0.4, color='r', label='Bottom radial linker force')
plt.hist(linker_f_rad_arr_all[:,isnare], bins=np.arange(-100,101,2), density=True, alpha=1, color='b', label='Total radial linker force')
#plt.axvline(x=-7, color='r', linestyle='--')
plt.xlabel('Force (pN)', fontdict=font_axis)
plt.ylabel('Probability density', fontdict=font_axis)
plt.title('Radial linker force', fontdict=font_title)
plt.xlim(-60, 60)
plt.ylim(0, 0.12)
#plt.legend()
plt.show()
'''

In [ ]:
# Plot the radial CTD positional distribution for one snare
'''
plt.hist(snare_ctd_pos_rad_arr_all[:,isnare], bins=np.arange(-10,10,0.1), density=True, alpha=1, color='g')
#plt.axvline(x=snare_ctd_pos_rad_equil_avg, color='r', linestyle='--')
plt.xlabel('r (nm)', fontdict=font_axis)
plt.ylabel('Probability density', fontdict=font_axis)
plt.title('Radial C-term position', fontdict=font_title)
plt.xlim(-1, 3)
plt.ylim(0, 1.7)
'''

In [ ]:
# Plot the radial linker force time trace for one snare
'''
plt.plot(t_plot, snare_f_rad_arr_all[:,isnare], color='g', label='Radial SNARE force')
#plt.plot(-(linker_up_f_rad_arr_all[:,isnare]), bins=np.arange(-100,101,2), density=True, alpha=0.4, color='b', label='Top radial linker force')
#plt.plot(-(linker_dn_f_rad_arr_all[:,isnare]), bins=np.arange(-100,101,2), density=True, alpha=0.4, color='r', label='Bottom radial linker force')
plt.plot(t_plot, (linker_f_rad_arr_all[:,isnare]), color='b', label='Total radial linker force')
#plt.plot(t_plot, -(linker_f_theta_arr_all[:,isnare]), color='r', label='Total tangential linker force')
plt.xlabel('Time (microsec)', fontdict=font_axis)
plt.ylabel('Force(pN)', fontdict=font_axis)
plt.title('Radial linker force', fontdict=font_title)
plt.xlim(0, 4)
plt.ylim(0,40)
#plt.ylim(-60,20)
#plt.legend()
plt.show()
'''